# Import Libraries

In [23]:
from langchain.llms.openai import OpenAIChat
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.router.llm_router import RouterOutputParser, LLMRouterChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains import (
    LLMChain,
    SimpleSequentialChain,
    SequentialChain,
    MultiPromptChain,
)
from langchain.globals import set_debug
from dotenv import load_dotenv

set_debug(True)  # Enable debug mode
load_dotenv()  # Load the variables from .env file

True

# LLMChain

Chain is an encapsulation of Model, prompt, and optionally a parser.

In [7]:
model = OpenAIChat(temperature=0.9)
prompt = PromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm=model, prompt=prompt)

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [9]:
chain("Potato Chips")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "product": "Potato Chips"
}
[llm/start] [1:chain:LLMChain > 2:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "What is the best name to describe     a company that makes Potato Chips?"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:OpenAIChat] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Crunch Factory",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 22,
      "completion_tokens": 3,
      "total_tokens": 25
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "The Crunch Factory"
}


{'product': 'Potato Chips', 'text': 'The Crunch Factory'}

# SimpleSequentialChain

Chains can be connected to each other. The output of the previous chain is the input of the next chain. The `SimpleSequentialChain` is a simple chain that can be used to connect multiple chains.

In [17]:
# This is a simpler method to create a template.
prompt_1 = PromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain_1 = LLMChain(llm=model, prompt=prompt_1)

In [12]:
prompt_2 = PromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

chain_2 = LLMChain(llm=model, prompt=prompt_2)

In [15]:
simple_chain = SimpleSequentialChain(chains=[chain_1, chain_2])

In [16]:
simple_chain("Potato Chips")

[chain/start] [1:chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "Potato Chips"
}
[chain/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "product": "Potato Chips"
}
[llm/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "What is the best name to describe     a company that makes Potato Chips?"
  ]
}
[llm/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] [1.90s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\"Crisp Delights\"",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 22,
      "completion_tokens": 6,
      "total_tokens": 28
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] [1.91s] Exiting Chain run with output:
{
  "text": "\"Crisp Delig

{'input': 'Potato Chips',
 'output': 'Crisp Delights offers a tantalizing selection of gourmet snacks and treats that are full of flavor and satisfaction.'}

# SequentialChain

it can be used to connect multiple chains non-sequentially, by specifying `output_key` that the next chain needs to receive.

In [36]:
# prompt template 1: translate to english
prompt_1 = PromptTemplate.from_template(
    "Translate the following review to english:" "\n\n{Review}"
)
# chain 1: input = Review and output = English_Review
chain_1 = LLMChain(llm=model, prompt=prompt_1, output_key="English_Review")

In [37]:
# prompt template 2: summarize
prompt_2 = PromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}"
)
# chain 2: input = English_Review and output = summary
chain_2 = LLMChain(llm=model, prompt=prompt_2, output_key="summary")

In [38]:
# prompt template 3: Know the language
prompt_3 = PromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input = Review and output = language
chain_3 = LLMChain(llm=model, prompt=prompt_3, output_key="language")

In [39]:
# prompt template 4: follow up message
prompt_4 = PromptTemplate.from_template(
    "Write a follow up response to the following \
    summary in the specified language: \
    \n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input = summary, language and output = followup_message
chain_4 = LLMChain(llm=model, prompt=prompt_4, output_key="followup_message")

In [40]:
# sequential_chain: input = Review
# and output = English_Review,summary, followup_message
sequential_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
)

In [41]:
french_customer_review = """\
Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. \
J'achète les mêmes dans le commerce et le goût est bien meilleur."
"""

In [43]:
sequential_response = sequential_chain(french_customer_review)

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "Review": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur.\"\n"
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "Review": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur.\"\n"
}
[llm/start] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Translate the following review to english:\n\nJe trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur.\""
  ]
}
[llm/end] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] [2.31s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I find the taste mediocre. The foam does not hold, 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


[llm/end] [1:chain:SequentialChain > 8:chain:LLMChain > 9:llm:OpenAIChat] [36.56s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Réponse de suivi:\n\nCher(e) critique,\n\nNous vous remercions d'avoir partagé vos commentaires sur votre expérience d'achat de notre produit. Nous sommes désolés d'apprendre que vous avez été déçu(e) par le goût médiocre et la faible stabilité de la mousse. Votre opinion est importante pour nous, et nous souhaitons nous améliorer en conséquence.\n\nNous tenons à souligner que nous prenons toutes les mesures nécessaires pour garantir la qualité de nos produits, y compris leur goût et leur texture. Cependant, il peut arriver que des variations se produisent en fonction de plusieurs facteurs, tels que la manipulation ou le stockage du produit.\n\nNous aimerions mieux comprendre votre expérience et identifier la cause de ce problème. Pourriez-vous nous fournir plus de détails, tels que le numéro de lot ou la date d'achat, afin 

In [45]:
sequential_response

{'Review': 'Je trouve le goût médiocre. La mousse ne tient pas, c\'est bizarre. J\'achète les mêmes dans le commerce et le goût est bien meilleur."\n',
 'English_Review': "I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better.",
 'summary': 'The reviewer was disappointed with the mediocre taste and lack of foam stability, contrasting it with a better taste experience from the same product purchased in stores.',
 'followup_message': "Réponse de suivi:\n\nCher(e) critique,\n\nNous vous remercions d'avoir partagé vos commentaires sur votre expérience d'achat de notre produit. Nous sommes désolés d'apprendre que vous avez été déçu(e) par le goût médiocre et la faible stabilité de la mousse. Votre opinion est importante pour nous, et nous souhaitons nous améliorer en conséquence.\n\nNous tenons à souligner que nous prenons toutes les mesures nécessaires pour garantir la qualité de nos produits, y compris leur goût et leur

# RouterChain

`RouterChain` is a chain that can be used to route the input to different chains based on the input key.

In [1]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [2]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [4]:
model = OpenAIChat(temperature=0.0)

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [6]:
# Creating the destination chains
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=model, prompt=prompt)
    destination_chains[name] = chain

In [26]:
# a default chain that will be used when the router is not sure which chain to use
default_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

In [9]:
# Creating the destination choices string
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [21]:
# the prompt template that will be used to route the input to the correct destination
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [22]:
# The router that decides which destination chain to use
router_chain = LLMRouterChain.from_llm(llm=model, prompt=router_prompt)

In [27]:
multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    default_chain=default_chain,
    destination_chains=destination_chains,
)

In [28]:
multi_prompt_chain("What is the speed of light?")

[chain/start] [1:chain:MultiPromptChain] Entering Chain run with input:
{
  "input": "What is the speed of light?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] Entering Chain run with input:
{
  "input": "What is the speed of light?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the speed of light?"
}
[llm/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to loo

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] [2.10s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"destination\": \"physics\",\n    \"next_inputs\": \"What is the speed of light?\"\n}\n```",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 276,
      "completion_tokens": 26,
      "total_tokens": 302
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] [2.10s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"destination\": \"physics\",\n    \"next_inputs\": \"What is the speed of light?\"\n}\n```"
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] [2.11s] Exiting Chain run with output:
{
  "destination": "physics",
  "next_inputs": {
    "input": "What is the speed of light?"
  }
}
[chain/start] [1:

{'input': 'What is the speed of light?',
 'text': 'The speed of light in a vacuum is approximately 299,792,458 meters per second, or about 186,282 miles per second.'}

you can see from the logs that the input is routed to the `physics` chain as the input topic is physics.

In [29]:
multi_prompt_chain("What is 1 + 1?")

[chain/start] [1:chain:MultiPromptChain] Entering Chain run with input:
{
  "input": "What is 1 + 1?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] Entering Chain run with input:
{
  "input": "What is 1 + 1?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is 1 + 1?"
}
[llm/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{\n    \"destination\

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] [2.39s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"destination\": \"math\",\n    \"next_inputs\": \"What is 1 + 1?\"\n}\n```",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 277,
      "completion_tokens": 27,
      "total_tokens": 304
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] [2.39s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"destination\": \"math\",\n    \"next_inputs\": \"What is 1 + 1?\"\n}\n```"
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] [2.39s] Exiting Chain run with output:
{
  "destination": "math",
  "next_inputs": {
    "input": "What is 1 + 1?"
  }
}
[chain/start] [1:chain:MultiPromptChain > 5:chain:LLMChain] Enter

{'input': 'What is 1 + 1?',
 'text': 'Thank you for your kind words! The question "What is 1 + 1?" is a simple addition problem. To solve it, we add the numbers 1 and 1 together. \n\n1 + 1 = 2\n\nTherefore, the answer to the question "What is 1 + 1?" is 2.'}

In [30]:
multi_prompt_chain("Who was the first president of the United States?")

[chain/start] [1:chain:MultiPromptChain] Entering Chain run with input:
{
  "input": "Who was the first president of the United States?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] Entering Chain run with input:
{
  "input": "Who was the first president of the United States?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "input": "Who was the first president of the United States?"
}
[llm/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\n

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] [2.95s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"destination\": \"History\",\n    \"next_inputs\": \"Who was the first president of the United States?\"\n}\n```",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 279,
      "completion_tokens": 29,
      "total_tokens": 308
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] [2.95s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"destination\": \"History\",\n    \"next_inputs\": \"Who was the first president of the United States?\"\n}\n```"
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] [2.95s] Exiting Chain run with output:
{
  "destination": "History",
  "next_inputs": {
    "input": "Who was 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


[llm/end] [1:chain:MultiPromptChain > 5:chain:LLMChain > 6:llm:OpenAIChat] [13.43s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The first president of the United States was George Washington. He served as the president from 1789 to 1797. Washington played a crucial role in the formation of the United States and is often referred to as the \"Father of His Country.\" He was a key figure in the American Revolutionary War and was unanimously elected as the first president under the newly established U.S. Constitution. Washington's leadership and contributions to the early years of the nation set important precedents for future presidents and helped shape the office of the presidency.",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 90,
      "completion_tokens": 105,
      "total_tokens": 195
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:MultiPromptCha

{'input': 'Who was the first president of the United States?',
 'text': 'The first president of the United States was George Washington. He served as the president from 1789 to 1797. Washington played a crucial role in the formation of the United States and is often referred to as the "Father of His Country." He was a key figure in the American Revolutionary War and was unanimously elected as the first president under the newly established U.S. Constitution. Washington\'s leadership and contributions to the early years of the nation set important precedents for future presidents and helped shape the office of the presidency.'}

In [31]:
multi_prompt_chain("explain the quicksort algorithm")

[chain/start] [1:chain:MultiPromptChain] Entering Chain run with input:
{
  "input": "explain the quicksort algorithm"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] Entering Chain run with input:
{
  "input": "explain the quicksort algorithm"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "input": "explain the quicksort algorithm"
}
[llm/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object form

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20

[llm/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] [17.58s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"destination\": \"computer science\",\n    \"next_inputs\": \"explain the quicksort algorithm\"\n}\n```",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 275,
      "completion_tokens": 26,
      "total_tokens": 301
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] [17.58s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"destination\": \"computer science\",\n    \"next_inputs\": \"explain the quicksort algorithm\"\n}\n```"
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] [17.58s] Exiting Chain run with output:
{
  "destination": "computer science",
  "next_inputs": {
    "input": "explain the qu

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.op

[llm/end] [1:chain:MultiPromptChain > 5:chain:LLMChain > 6:llm:OpenAIChat] [37.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The quicksort algorithm is a widely used sorting algorithm that follows the divide-and-conquer approach. It efficiently sorts an array or list by partitioning it into smaller sub-arrays, sorting those sub-arrays recursively, and then combining them to obtain a sorted array.\n\nHere is a step-by-step explanation of the quicksort algorithm:\n\n1. Choose a pivot element from the array. The pivot can be any element, but commonly the first or last element is chosen.\n\n2. Partition the array into two sub-arrays: elements less than the pivot and elements greater than the pivot. This is done by iterating through the array and comparing each element to the pivot. If an element is smaller than the pivot, it is moved to the left sub-array, otherwise to the right sub-array.\n\n3. Recursively apply steps 1 and 2 to the sub-arrays creat

{'input': 'explain the quicksort algorithm',
 'text': 'The quicksort algorithm is a widely used sorting algorithm that follows the divide-and-conquer approach. It efficiently sorts an array or list by partitioning it into smaller sub-arrays, sorting those sub-arrays recursively, and then combining them to obtain a sorted array.\n\nHere is a step-by-step explanation of the quicksort algorithm:\n\n1. Choose a pivot element from the array. The pivot can be any element, but commonly the first or last element is chosen.\n\n2. Partition the array into two sub-arrays: elements less than the pivot and elements greater than the pivot. This is done by iterating through the array and comparing each element to the pivot. If an element is smaller than the pivot, it is moved to the left sub-array, otherwise to the right sub-array.\n\n3. Recursively apply steps 1 and 2 to the sub-arrays created in the previous step. This means choosing a pivot for each sub-array and partitioning them further until th

In [32]:
multi_prompt_chain("What is the functions of the heart?")

[chain/start] [1:chain:MultiPromptChain] Entering Chain run with input:
{
  "input": "What is the functions of the heart?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] Entering Chain run with input:
{
  "input": "What is the functions of the heart?"
}
[chain/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the functions of the heart?"
}
[llm/start] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[llm/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain > 4:llm:OpenAIChat] [1.52s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{\n    \"destination\": \"DEFAULT\",\n    \"next_inputs\": \"What is the functions of the heart?\"\n}\n```",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 277,
      "completion_tokens": 27,
      "total_tokens": 304
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain > 3:chain:LLMChain] [1.52s] Exiting Chain run with output:
{
  "text": "```json\n{\n    \"destination\": \"DEFAULT\",\n    \"next_inputs\": \"What is the functions of the heart?\"\n}\n```"
}
[chain/end] [1:chain:MultiPromptChain > 2:chain:LLMRouterChain] [1.52s] Exiting Chain run with output:
{
  "destination": null,
  "next_inputs": {
    "input": "What is the functions of the heart?"
  }


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


[llm/end] [1:chain:MultiPromptChain > 5:chain:LLMChain > 6:llm:OpenAIChat] [22.51s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The heart is a vital organ that performs several important functions in the body. Some of the main functions of the heart include:\n\n1. Pumping blood: The heart acts as a muscular pump that continuously circulates blood throughout the body. It receives oxygenated blood from the lungs and pumps it to the rest of the body, delivering oxygen and nutrients to the cells and organs.\n\n2. Oxygenation: The heart receives deoxygenated blood from the body and pumps it to the lungs, where it picks up oxygen and gets rid of carbon dioxide. This oxygenation process ensures that the body's cells receive the necessary oxygen for their proper functioning.\n\n3. Regulation of blood pressure: The heart helps regulate blood pressure by adjusting the force and rate of its contractions. It pumps blood with enough force to maintain adequate bl

{'input': 'What is the functions of the heart?',
 'text': "The heart is a vital organ that performs several important functions in the body. Some of the main functions of the heart include:\n\n1. Pumping blood: The heart acts as a muscular pump that continuously circulates blood throughout the body. It receives oxygenated blood from the lungs and pumps it to the rest of the body, delivering oxygen and nutrients to the cells and organs.\n\n2. Oxygenation: The heart receives deoxygenated blood from the body and pumps it to the lungs, where it picks up oxygen and gets rid of carbon dioxide. This oxygenation process ensures that the body's cells receive the necessary oxygen for their proper functioning.\n\n3. Regulation of blood pressure: The heart helps regulate blood pressure by adjusting the force and rate of its contractions. It pumps blood with enough force to maintain adequate blood flow and pressure throughout the body.\n\n4. Circulatory system support: The heart is the central orga